In [6]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


In [7]:
## We will be seeing image pyramids here.
## normally we are using images of fixed size. However, sometimes, we need to use images of 
## different sizes - For Example: when we need to capture an object with unknown size
## there are two pyramids available in OpenCV - laplacian pyramid and Gaussian pyramid
## lets start with Gaussian - It is a process of repeat filtering and subsampling
## it is further divided into paired down and paired up - lets start with paired down
## first thing - need to read the image
img = cv.imread("./Lena_Copy.png", -1)
print("IMAGE", img.shape)
## to apply paired down:
pyrDwn = cv.pyrDown(img, ) ## reduces the size to 1/fourth with no arguments provided
print("PyrDwn", pyrDwn.shape)
## further reducing the size
pyrDwn_2 = cv.pyrDown(pyrDwn)
print("pyrDwn_2", pyrDwn_2.shape)


## next, lets see pyrUp method:
pyrUp = cv.pyrUp(img)
pyrUp_2 = cv.pyrUp(pyrDwn_2)
print("pyrUp", pyrUp.shape)

print("pyrUp_2", pyrUp_2.shape)
## lets display them

cv.imshow("IMAGE", img)
cv.imshow("PyrDown", pyrDwn)
cv.imshow("PyrDown_2", pyrDwn_2)
cv.imshow("PyrUp", pyrUp)
cv.imshow("PyrUp_2", pyrUp_2)

cv.waitKey(0)
cv.destroyAllWindows()

IMAGE (512, 512, 3)
PyrDwn (256, 256, 3)
pyrDwn_2 (128, 128, 3)
pyrUp (1024, 1024, 3)
pyrUp_2 (256, 256, 3)


In [8]:
## next, if we want to create multi-resolution images without using pyrUp, pyrDown repeatedy,
## we can do it by using a simple for loop
## re-using the above code
img = cv.imread("./Lena_Copy.png", -1)
imgCopy = img.copy()

# creating gaussian pyramid array
gausPyr = [imgCopy]

## lets create six different resolution images
for i in range(6):
    imgCopy = cv.pyrDown(imgCopy)
    # gausPyr.append(imgCopy)
    cv.imshow("IMAGE:{}".format(i), imgCopy)
    


cv.imshow("Original IMAGE", img)

cv.waitKey(0)
cv.destroyAllWindows()
## similarly we can use for loop to create pyrUp images

In [9]:
## lets look into laplacian pyramids - they are basically formed from gausian pyramids
## there is no specific function designed to create the laplacian pyramids
## they are created by using the difference of gausian pyramids and their expanded versions of 
## upper layers gaussian pyramids
## lets understand it with the help of the code
## re-using the above code
## here the top level is the last generated image
img = cv.imread("./Lena_Copy.png", -1)
imgCopy = img.copy()

# creating gaussian pyramid array
gausPyr = [imgCopy]

## lets create six different resolution images
for i in range(6):
    imgCopy = cv.pyrDown(imgCopy)
    gausPyr.append(imgCopy)
    # cv.imshow("IMAGE:{}".format(i), imgCopy)
    

## lets get the last generated image
uprlvl = gausPyr[-1]

## lets create a laplacian pyramid as we created gaussian pyramid
lapPyr = [uprlvl]

## similarly using the for loop:
for i in range(5,0, -1):
    ## as laplacian pyramid is the difference between upper extended gaussian versions and s
    ## simple gaussian versions - lets find both of them
    extend_gaus = cv.pyrUp(gausPyr[i])
    laplac_img = cv.subtract(gausPyr[i-1], extend_gaus)
    cv.imshow("Laplacian Image: {}".format(i), laplac_img)
    
cv.imshow("Original IMAGE", img)
cv.imshow("Upper Level", uprlvl)

cv.waitKey(0)
cv.destroyAllWindows()


In [10]:
## next we are going to see image blending 
## first, lets read the image
apple = cv.imread("./apple.jpg", -1)
orange = cv.imread("./orange.jpg", -1)
print(apple.shape)
print(orange.shape)

## next lets see how to blend or merge these two images - first lets start with the blending
## first lets get it the easy way of cutting the parts which we want to blend and stacked them
## for that we can use the np.hstack() method
app_orn = np.hstack((apple[:, :256], orange[:, 256:]))

## here we can see clearly the differentiating line - in image blending we need to 
## properly blend them - for that we can use image pyramid techniques
## this can be done in total of five steps: we have already done first step of loading images
## Second step  - finding the gausian pyramid for apple and orange
## lets repeat the above steps
apple_copy = apple.copy()
## gausain pyramid array
gp_apple = [apple_copy]

## using for loop to create apple gausian pyramids
for i in range(6):
    apple_copy = cv.pyrDown(apple_copy)
    gp_apple.append(apple_copy)

## similarly repeating the process for orage
orange_copy = orange.copy()
gp_orange = [orange_copy]
for i in range(6):
    orange_copy = cv.pyrDown(orange_copy)
    gp_orange.append(orange_copy)

    
## step 3 - creating the laplacian pyramid for apple and orange
## for apple:
apple_copy = gp_apple[5]
lp_apple = [apple_copy]

for i in range(5, 0, -1):
    gaussian_extended = cv.pyrUp(gp_apple[i])
    laplacian = cv.subtract(gp_apple[i-1], gaussian_extended)
    lp_apple.append(laplacian)

## similarly repeating the step for orange
orange_copy = gp_orange[5]
lp_orange = [orange_copy]

for i in range(5, 0, -1):
    gaussian_extended = cv.pyrUp(gp_orange[i])
    laplacian = cv.subtract(gp_orange[i-1], gaussian_extended)
    lp_orange.append(laplacian)
## Step 4 - Joining half of both images
## starting by creating an empty list to be used later to append the images
apple_orange_pyramid = []
n=0
for apple_lap, orange_lap in zip(lp_apple, lp_orange):
    n += 1
    col, rows, ch = apple_lap.shape
    laplacian = np.hstack((apple_lap[:, 0:int(col/2)], orange_lap[:, int(col/2):]))
    apple_orange_pyramid.append(laplacian)

## final step is to reconstruct all the images
apple_orang_reconstruct = apple_orange_pyramid[0]
for i in range(1, 6):
    apple_orang_reconstruct = cv.pyrUp(apple_orang_reconstruct)

    apple_orang_reconstruct = cv.add(apple_orange_pyramid[i], apple_orang_reconstruct)

    
## lets display them

cv.imshow("APPLE", apple)
cv.imshow("ORANGE", orange)
cv.imshow("Apple&Orange", app_orn)
cv.imshow("Apple&OrangeReconstructed", apple_orang_reconstruct)
cv.waitKey(0)
cv.destroyAllWindows()

(512, 512, 3)
(512, 512, 3)
